In [ ]:
!pip install tokenizers==0.9.4

In [ ]:
pip install transformers -U

Requirement already up-to-date: transformers in /usr/local/lib/python3.7/dist-packages (4.6.0)
     |████████████████████████████████| 3.3MB 5.7MB/s 
  Found existing installation: tokenizers 0.9.4
    Uninstalling tokenizers-0.9.4:
      Successfully uninstalled tokenizers-0.9.4


In [ ]:
!pip install simpletransformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import torchvision
import os
from torchvision import transforms, utils
from PIL import Image
from torchvision.transforms import ToTensor
from torch.autograd import Variable
import torch
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler
import matplotlib.pyplot as plt
import PIL 
from torch import nn
import time
import random
from matplotlib import gridspec
from simpletransformers.classification import ClassificationModel
from sklearn.model_selection import train_test_split

In [ ]:
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.enabled = False 
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

In [ ]:
set_seed(7)

In [ ]:
def removeDuplicates(S): 
          
    n = len(S)  
      
    # We don't need to do anything for  
    # empty or single character string.  
    if (n < 2) : 
        return
          
    # j is used to store index is result  
    # string (or index of current distinct  
    # character)  
    j = 0
      
    # Traversing string  
    for i in range(n):  
          
        # If current character S[i]  
        # is different from S[j]  
        if (S[j] != S[i]): 
            j += 1
            S[j] = S[i]  
      
    # Putting string termination  
    # character.  
    j += 1
    S = S[:j] 
    return "".join(S) 

In [ ]:
train=pd.read_csv("/content/drive/MyDrive/pcd/Train.csv")
test=pd.read_csv("/content/drive/MyDrive/pcd/Test.csv")
train.head()

,ID,text,label
0,SKCLXCJ,cha3eb fey9elkoum menghir ta7ayoul ou kressi,-1
1,V1TVXIJ,bereau degage nathef ya slim walahi ya7chiw fi...,-1
2,U0TTYY8,ak slouma,1
3,68DX797,entom titmanou lina a7na 3iid moubarik a7na ch...,-1
4,OSTX8NX,hhhhhhhh blidaa minik ba3d doussieet athika il...,1


In [ ]:
train["text"]=train['text'].apply(lambda x :removeDuplicates(list(x.rstrip())) )

In [ ]:
train.shape

(70000, 3)

In [ ]:
train.label.value_counts()

 1    38239
-1    29295
 0     2466
Name: label, dtype: int64

In [ ]:
train.label+=1

In [ ]:
train['text'].apply(lambda x: len(x)).describe()

count    70000.000000
mean        54.924743
std         84.292720
min          3.000000
25%         19.000000
50%         33.000000
75%         62.000000
max       9629.000000
Name: text, dtype: float64

In [ ]:
train.isnull().sum()

ID       0
text     0
label    0
dtype: int64

In [ ]:
test['text'].apply(lambda x: len(x)).describe()

count    30000.000000
mean        60.031067
std         92.991510
min          3.000000
25%         21.000000
50%         36.000000
75%         67.000000
max       6270.000000
Name: text, dtype: float64

In [ ]:
test["text"]=test['text'].apply(lambda x :removeDuplicates(list(x.rstrip())) )

In [ ]:
test.head()

,ID,text
0,2DDHQW9,barcha aindou fiha hak w barcha teflim kadhalik
1,5HY6UEY,ye gernabou ye 9a7ba
2,ATNVUJX,saber w bara rabi m3ak 5ouya
3,Q9XYVOQ,cha3éb ta7an tfouh
4,TOAHLRH,rabi y5alihoulek w yfar7ek bih w inchalah itch...


In [ ]:
def get_model(model_type, model_name, n_epochs = 2, train_batch_size = 100, eval_batch_size = 64, seq_len = 120, lr = 2e-5):
  model = ClassificationModel(model_type, model_name,num_labels=3, args={'train_batch_size':train_batch_size,
                                                                         "eval_batch_size": eval_batch_size,
                                                                         'reprocess_input_data': True,
                                                                         'overwrite_output_dir': True,
                                                                         'fp16': False,
                                                                         'do_lower_case': False,
                                                                         'num_train_epochs': n_epochs,
                                                                         'max_seq_length': seq_len,
                                                                         'manual_seed': 2,
                                                                         "learning_rate":lr,
                                                                         "save_eval_checkpoints": False,
                                                                         "save_model_every_epoch": False,})
  return model

In [ ]:
tmp = pd.DataFrame()
tmp['text'] = train['text']
tmp['labels'] = train['label']


tmp_trn, tmp_val = train_test_split(tmp, test_size=0.1, random_state=2)


In [ ]:

if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: Tesla T4


In [ ]:
model = get_model('distilbert', 'distilbert-base-uncased', n_epochs=2,lr=2e-4,seq_len=150,train_batch_size=160)


Downloading:   0%|          | 0.00/442 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'pre_classi

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [ ]:
model.train_model(tmp_trn)


  0%|          | 0/63000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/394 [00:00<?, ?it/s]

Running Epoch 1 of 2:   0%|          | 0/394 [00:00<?, ?it/s]

(788, 0.45780419987546006)

In [ ]:
from sklearn.metrics import f1_score, accuracy_score

def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')

In [ ]:
#preds_val = model.eval_model(tmp_val,acc=accuracy_score)
result, model_outputs, wrong_predictions = model.eval_model(tmp_val, f1=f1_multiclass, acc=accuracy_score)


  0%|          | 0/7000 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/110 [00:00<?, ?it/s]

In [ ]:
result

{'acc': 0.819,
 'eval_loss': 0.4490079316225919,
 'f1': 0.819,
 'mcc': 0.654097241818579}

In [ ]:
predictions, raw_outputs =model.predict(['ya bhim','yaatik esaha'])

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
predictions -=1
predictions

array([-1,  1])

In [ ]:
from scipy.special import softmax


In [ ]:
pred=model.predict(list(test['text']))

  0%|          | 0/30000 [00:00<?, ?it/s]

  0%|          | 0/469 [00:00<?, ?it/s]

In [ ]:
probs=softmax(pred[1],axis=1)

In [ ]:
test_data = test['text']
test_id = test['ID']

dataf = pd.DataFrame(data = {'id' : test_id ,'text': test['text'], 'negative' : probs[:,0] ,'neutral' : probs[:,1] ,'positive' : probs[:,2]  })
dataf.to_csv('/content/drive/MyDrive/pcd/resultbert.csv')